# Marking imputed values

In this recipe, we will add binary variables to indicate that a value was missing using pandas, Scikit-learn and Feature-Engine.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from feature_engine.imputation import (
    AddMissingIndicator,
    CategoricalImputer,
    MeanMedianImputer,
)

## Load data

In [2]:
data = pd.read_csv("credit_approval_uci.csv")

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,NaN,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,NaN,NaN,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


## Split data into train and test sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("target", axis=1),
    data["target"],
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((483, 15), (207, 15))

## Add missing indicator with pandas

In [4]:
# Capture variable names in a list:

varnames = ["A1", "A3", "A4", "A5", "A6", "A7", "A8"]

In [5]:
# Create missing indicator names in a list:

indicators = [f"{var}_na" for var in varnames]

indicators

['A1_na', 'A3_na', 'A4_na', 'A5_na', 'A6_na', 'A7_na', 'A8_na']

In [6]:
# Add missing indicators:

X_train_t = X_train.copy()
X_test_t = X_test.copy()

X_train_t[indicators] = X_train[varnames].isna().astype(int)
X_test_t[indicators] = X_test[varnames].isna().astype(int)

In [7]:
# Check the new missing indicator variables:

X_train_t.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A13,A14,A15,A1_na,A3_na,A4_na,A5_na,A6_na,A7_na,A8_na
596,a,46.08,3.000,u,g,c,v,2.375,NaN,t,...,g,396.0,4159,0,0,0,0,0,0,0
303,a,15.92,2.875,u,g,q,v,0.085,NaN,f,...,g,120.0,0,0,0,0,0,0,0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,...,g,50.0,1187,0,0,0,0,0,0,0
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,...,g,100.0,0,0,0,0,0,0,0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,...,g,360.0,1332,0,0,0,0,0,0,0


In [8]:
# The mean of the missing indicators should be the same as the
# fraction of missing values in the original variable.

# Let's check that out for the variable A3:

X_train["A3"].isnull().mean(), X_train_t["A3_na"].mean()

(0.14078674948240166, 0.14078674948240166)

## Adding missing indicator with Feature-engine

In [9]:
# Set up imputer to add indicators to variables
# with missing data:

imputer = AddMissingIndicator(
    variables=None,
    missing_only=True,
)

# Imputer finds variables with missing data:

imputer.fit(X_train)

AddMissingIndicator()

In [10]:
# The variables that had missing data in
# the train set:

imputer.variables_

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A14']

In [11]:
# Add missing indicators:

X_train_t = imputer.transform(X_train)
X_test_t = imputer.transform(X_test)

In [12]:
# Check the new missing indicator variables:

X_train_t.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A2_na,A3_na,A4_na,A5_na,A6_na,A7_na,A8_na,A9_na,A10_na,A14_na
596,a,46.08,3.000,u,g,c,v,2.375,NaN,t,...,0,0,0,0,0,0,0,1,0,0
303,a,15.92,2.875,u,g,q,v,0.085,NaN,f,...,0,0,0,0,0,0,0,1,0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,...,0,0,0,0,0,0,0,0,0,0
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,...,0,0,0,0,0,0,0,0,0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,...,0,0,0,0,0,0,0,0,0,0


### Add missing indicators and impute data

In [13]:
# Let's create a pipeline, where we add missing indicators
# first, and then impute categorical variables with the
# frequent category, and numerical variables with the mean:

pipe = Pipeline(
    [
        ("indicators", AddMissingIndicator(missing_only=True)),
        ("categorical", CategoricalImputer(imputation_method="frequent")),
        ("numerical", MeanMedianImputer()),
    ]
)

In [14]:
# Add indicators and impute data:

X_train_t = pipe.fit_transform(X_train)
X_test_t = pipe.transform(X_test)

In [15]:
# Final dataset after the imputation:

X_train_t.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A2_na,A3_na,A4_na,A5_na,A6_na,A7_na,A8_na,A9_na,A10_na,A14_na
596,a,46.08,3.000,u,g,c,v,2.375,t,t,...,0,0,0,0,0,0,0,1,0,0
303,a,15.92,2.875,u,g,q,v,0.085,t,f,...,0,0,0,0,0,0,0,1,0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,...,0,0,0,0,0,0,0,0,0,0
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,...,0,0,0,0,0,0,0,0,0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Corroborate absence of missing values:

X_train.isnull().sum()

A1      4
A2     11
A3     68
A4      4
A5      4
A6      4
A7      4
A8     64
A9     69
A10    66
A11     0
A12     0
A13     0
A14     7
A15     0
dtype: int64

## Adding missing indicator with Scikit-learn

In [17]:
indicator = MissingIndicator(
    features="missing-only").set_output(transform="pandas")

indicator.fit(X_train)

MissingIndicator()

In [18]:
# The features that had missing data in the train set
# (the NumPy array shows the column index):

indicator.features_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 13], dtype=int64)

In [19]:
# Concatenate the original dataset with the missing indicators
X_train_t = pd.concat(
    [
        X_train.reset_index(drop=True),
        indicator.transform(X_train),
    ],
    axis=1,
)

X_train_t.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,missingindicator_A2,missingindicator_A3,missingindicator_A4,missingindicator_A5,missingindicator_A6,missingindicator_A7,missingindicator_A8,missingindicator_A9,missingindicator_A10,missingindicator_A14
0,a,46.08,3.000,u,g,c,v,2.375,NaN,t,...,False,False,False,False,False,False,False,False,False,False
1,a,15.92,2.875,u,g,q,v,0.085,NaN,f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b,36.33,2.125,y,p,w,v,0.085,t,t,...,False,True,False,False,False,False,False,False,False,False
3,b,22.17,0.585,y,p,ff,ff,0.000,f,f,...,False,False,False,False,False,False,False,True,True,False
4,b,57.83,7.040,u,g,m,v,14.000,t,t,...,False,False,False,False,False,False,False,False,False,False


### Add missing indicators and impute data

In [20]:
numvars = X_train.select_dtypes(exclude="O").columns.to_list()
catvars = X_train.select_dtypes(include="O").columns.to_list()

In [21]:
# Pipeline:

pipe = ColumnTransformer(
    [
        ("num_imputer", SimpleImputer(
            strategy="mean", add_indicator=True), numvars),
        ("cat_imputer", SimpleImputer(
            strategy="most_frequent", add_indicator=True), catvars),
    ]
).set_output(transform="pandas")

In [22]:
# Add indicators and impute data:

X_train_t = pipe.fit_transform(X_train)
X_test_t = pipe.transform(X_test)

In [23]:
# The result is a Numpy array:

X_train_t.head()

,num_imputer__A2,num_imputer__A3,num_imputer__A8,num_imputer__A11,num_imputer__A14,num_imputer__A15,num_imputer__missingindicator_A2,num_imputer__missingindicator_A3,num_imputer__missingindicator_A8,num_imputer__missingindicator_A14,...,cat_imputer__A10,cat_imputer__A12,cat_imputer__A13,cat_imputer__missingindicator_A1,cat_imputer__missingindicator_A4,cat_imputer__missingindicator_A5,cat_imputer__missingindicator_A6,cat_imputer__missingindicator_A7,cat_imputer__missingindicator_A9,cat_imputer__missingindicator_A10
596,46.08,3.000,2.375,8.0,396.0,4159.0,0.0,0.0,0.0,0.0,...,t,t,g,False,False,False,False,False,True,False
303,15.92,2.875,0.085,0.0,120.0,0.0,0.0,0.0,0.0,0.0,...,f,f,g,False,False,False,False,False,True,False
204,36.33,2.125,0.085,1.0,50.0,1187.0,0.0,0.0,0.0,0.0,...,t,f,g,False,False,False,False,False,False,False
351,22.17,0.585,0.000,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,f,f,g,False,False,False,False,False,False,False
118,57.83,7.040,14.000,6.0,360.0,1332.0,0.0,0.0,0.0,0.0,...,t,t,g,False,False,False,False,False,False,False
